In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import math
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.utils import class_weight
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.models import load_model
import joblib
# import py_vollib.black_scholes.implied_volatility
import multiprocessing
from multiprocessing import Pool
import os
import py_vollib.black_scholes.implied_volatility as vb
import statistics
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Path settings


path_var = 1 

if (path_var == 1):
    filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'
    modelpath = '/home/docboy/Desktop/Option_Pricing/pickles/'


else:
    filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'
    mdoelpath = '/home/sharan/Desktop/Option_Pricing/pickles/'



nfty = filepath + 'NIFTY50_feature_set_COV_complete.csv'
nfty_ecdf = filepath + 'NIFTY50_feature_set_ECDF_complete.csv'
    
nfty_19 = filepath + 'NIFTY50_2019_feature_set_COV_complete.csv'
nfty_ecdf_19 = filepath + 'NIFTY50_2019_feature_set_ECDF_complete.csv'


In [3]:
# Set the automated variables + Plus drop all useless columns approach specific


###########################
###############################################

approach = 3                      #------------------>>> # Change value as needed

###############################################
###########################


if (approach == 1):
    feats = pd.read_csv(nfty_ecdf)
    print ('Approach var set to ECDF')
    
    dropNames = ['date', 'percentage','strike','spot','opPrice']
    
    test_feats = pd.read_csv(nfty_ecdf_19)
    
else:
    feats = pd.read_csv(nfty)
    print ('Approach var set to :: '+ str(approach))
    
    test_feats = pd.read_csv(nfty_19)
    
    if (approach == 2):
        dropNames= ['date', 'percentage','strike','spot','opPrice','change_oi','ts_atr','prevOp',
                    'avg_atmError']
        
    else:
        dropNames= ['date', 'percentage','strike','spot','opPrice','ts_atr','change_oi']
        
###################




X=feats.iloc[:,:-1]
Y=feats.loc[:,'Target']
X = X.drop(dropNames, axis = 1)    





G = test_feats.iloc[:,:-1]
H = test_feats.loc[:,'Target']
G = G.drop(dropNames, axis = 1)  


# Port part handled at the end


Approach var set to :: 3


In [4]:
X.columns

Index(['close_mean', 'close_sd', 'open_mean', 'open_sd', 'high_mean',
       'high_sd', 'low_mean', 'low_sd', 'close_lr_open_lr_cov',
       'close_lr_high_lr_cov', 'close_lr_low_lr_cov', 'open_lr_high_lr_cov',
       'open_lr_low_lr_cov', 'high_lr_low_lr_cov', 'avg_atmError', 'prevOp',
       'ttm', 'atmError', 'yield03'],
      dtype='object')

In [5]:
# Start the scaling process

featNames = list(X)  
scaler = RobustScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns = featNames)




G = scaler.transform(G)
G = pd.DataFrame(G, columns = featNames)


In [6]:
# # # Save the scalar file
# joblib.dump(scaler, modelpath + 'DL_SCALER_NIFTY_0.02_Expt' + str(approach) + '.model')
# print("Scaler saved to path ::", modelpath + 'DL_SCALER_NIFTY_0.02_Expt' + str(approach) + '.model' )

In [7]:
onehot = OneHotEncoder(sparse=False)
Y = Y.values
Y = Y.reshape(len(Y), 1)
Y = onehot.fit_transform(Y)


onehot = OneHotEncoder(sparse=False)
H = H.values
H = H.reshape(len(H), 1)
H = onehot.fit_transform(H)




In [9]:
import datetime
feats['date'] = pd.to_datetime(feats['date'])
test=feats[(feats['date']>=datetime.datetime(2017,10,1))]
split_val = test.index[0]

X_train = X.iloc[0:split_val, :]
# X_test = X.iloc[split_val:, :]
Y_train = Y[:split_val]
# # Y_test = Y[split_val:]
# print ('X & Y_Dim of Train is', X_train.shape, Y_train.shape)
# print ('X & Y_Dim of Test is', X_test.shape, Y_test.shape)


# X_train = X


X_test = G


# Y_train = Y
Y_test = H



print ('X & Y_Dim of Train is', X_train.shape, Y_train.shape)
print ('X & Y_Dim of Test is', X_test.shape, Y_test.shape)





X & Y_Dim of Train is (10837, 19) (10837, 50)
X & Y_Dim of Test is (7667, 19) (7667, 50)


In [24]:
opt=optimizers.Adam(lr=0.00018);
model = keras.models.Sequential()
model.add(keras.layers.Dense(units=128, activation='relu',input_dim=X_train.shape[1]))
# model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(units=128, activation='relu'))
# model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units=64, activation='relu'))
# model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units=50, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy'],)
history=model.fit(X_train, Y_train,
  batch_size=32,
  epochs=40,
  verbose=1,
#   callbacks=[EarlyStopping(monitor='val_loss', patience=1,min_delta=-0.01)],
#   class_weight=class_weights,
#   shuffle=True,                
  validation_data=(X_test, Y_test))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               2560      
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                3250      
Total params: 14,066
Trainable params: 14,066
Non-trainable params: 0
_________________________________________________________________
Train on 10837 samples, validate on 7667 samples
Epoch 1/40
10837/10837 [==============================] - 2s 207us/step - loss: 3.7990 - accuracy: 0.0576 - val_loss: 3.5659 - val_accuracy: 0.1071
Epoch 2/40
10837/10837 [==============================] - 2s 205us/step - loss: 3.3929 - accuracy: 0.0737 - val_loss: 3.1411 - val_accuracy: 0.1183
Epoch 3/40
10837/10837 [===========

KeyboardInterrupt: 

In [ ]:
if(path_var ==1):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

elif(path_var==2):

    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', score[1]*100)

Y_pred=model.predict(X_test)
Y_pred=np.argmax(Y_pred,axis=1)

In [ ]:
OGY_test=onehot.inverse_transform(Y_test)
# OGY_test=sum(OGY_test, [])
OGY_test=OGY_test.ravel()
# print(OGY_test)

# # Check if Binwidth value exists
maxVal=5
minVal=0

width = maxVal - minVal

num_bins = 50


bin_width = width/num_bins
# print(bin_width)

# Normalized Error Metric (NEM)
diff = []
dist_sum = 0
for i in range(len(Y_pred)):
    dist = abs(Y_pred[i] - OGY_test[i])
    d = Y_pred[i] - OGY_test[i]
    diff.append(d)
    dist_sum = dist_sum + dist
    
print((dist_sum*bin_width)/len(Y_test))

In [ ]:
# u = {'C_i' : OGY_test, 'P_i' : Y_pred, 'C_i-P_i': diff}
# g = pd.DataFrame(u)
# g.to_csv(filepath + 'ANN_Cipi.csv', index = False)


In [18]:
def c_int(lst):
    df = pd.DataFrame(lst)
    df = df.sort_values(by = 0).reset_index(drop=True)
    cp1 = df[np.logical_and(df[0] >= -2,df[0] <= 2)]
    return (len(cp1)/len(df))*100
   
# lst = Y_test - k
# c_int(lst)

In [19]:
CP=[]
for i in range(len(Y_pred)):
    dist = OGY_test[i] - Y_pred[i]
    CP.append(dist)
CP=np.array(CP,dtype=float)

print("Confidence Interval:{}".format(c_int(CP)))
# np.savetxt(filepath+"CPNIFTY.csv",CP, delimiter=",")

Confidence Interval:63.49289161340811


In [15]:
# # Saving the model
# joblib.dump(model, modelpath + 'DL_NIFTY_0.02_Expt' + str(approach) + '.model')
# print("Model saved to path ::", modelpath + 'DL_NIFTY_0.02_Expt' + str(approach) + '.model' )

Model saved to path :: /home/sharan/Desktop/OptionPricing-master/pickles/DL_NIFTY_0.02_Expt1.model


In [16]:
# from sklearn.metrics import confusion_matrix,classification_report
# print(classification_report(OGY_test, Y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.79       197
           1       0.38      0.41      0.39       160
           2       0.26      0.34      0.29       132
           3       0.19      0.13      0.15       118
           4       0.12      0.12      0.12       107
           5       0.08      0.03      0.04       104
           6       0.16      0.27      0.20        93
           7       0.17      0.24      0.20        90
           8       0.22      0.09      0.13        78
           9       0.21      0.07      0.11        84
          10       0.16      0.40      0.23        70
          11       0.50      0.01      0.03        68
          12       0.20      0.29      0.24        58
          13       0.21      0.27      0.24        63
          14       0.18      0.05      0.08        62
          15       0.12      0.12      0.12        50
          16       0.18      0.25      0.21        55
          17       0.23    

/home/sharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Port Part

In [17]:
# b_feats['date'] = pd.to_datetime(b_feats['date'])
# b_test=b_feats[(b_feats['date']>=datetime.date(2017,10,1))]
# ind = b_test.index[0]

# b_X = b_feats.iloc[ind:,:-1]
# b_Y = b_feats.loc[ind:,'Target']
# b_X = b_X.drop(dropNames, axis = 1)

# featNames = list(b_X)
# b_X = scaler.transform(b_X)
# b_X = pd.DataFrame(b_X, columns = featNames)

# print("Test Size={}".format(len(b_Y)))

# # Generate Class weights

# class_weights = class_weight.compute_class_weight('balanced',np.unique(b_Y),b_Y)


# # Port the Onehot tranform
# b_Y = b_Y.values
# b_Y = b_Y.reshape(len(b_Y), 1)
# b_Y = onehot.transform(b_Y)




# # Split the train and test sets without random optimizations


# b_X_test = b_X.iloc[:, :]
# b_Y_test = b_Y[:]
# print ('b_X & b_Y_Dim of Test is', b_X_test.shape, b_Y_test.shape)

Test Size=6792
b_X & b_Y_Dim of Test is (6792, 22) (6792, 50)


In [18]:
# score = model.evaluate(b_X_test, b_Y_test, verbose=0)
# print('Test accuracy:', score[1]*100)

# b_Y_pred=model.predict(b_X_test)
# b_Y_pred=np.argmax(b_Y_pred,axis=1)

Test accuracy: 24.263839811542994


In [19]:
# b_OGY_test = onehot.inverse_transform(b_Y_test)
# b_OGY_test = b_OGY_test.ravel()

# # Check if Binwidth value exists
# maxVal=5
# minVal=0

# width = maxVal - minVal

# num_bins = 50


# bin_width = width/num_bins
# # print(bin_width)

# # Normalized Error Metric (NEM)

# dist_sum = 0
# for i in range(len(b_Y_pred)):
#     dist = abs(b_Y_pred[i] - b_OGY_test[i])
#     dist_sum = dist_sum + dist
    
# print((dist_sum*bin_width)/len(b_Y_test))

0.18938457008244997


In [20]:
# CP=[]
# for i in range(len(b_Y_pred)):
#     dist = b_OGY_test[i] - b_Y_pred[i]    
#     CP.append(dist)
# CP=np.array(CP,dtype=float)

# print("Confidence Interval:{}".format(c_int(CP)))

Confidence Interval:71.52532391048292
